In [1]:
# Running %env without any arguments
# lists all environment variables

# The line below sets the environment
# variable CUDA_VISIBLE_DEVICES
%env CUDA_VISIBLE_DEVICES = 

import numpy as np
import pandas as pd
import io
import time
import bson                       # this is installed with the pymongo package
import matplotlib.pyplot as plt
from scipy.misc import imread, imsave
import tensorflow as tf
from tensorflow.python.platform import tf_logging
from tensorflow.contrib import layers
from tensorflow.contrib.training import add_gradients_summaries
from tensorflow.python.ops import math_ops
from tensorflow.python.framework import ops
from tensorflow.python.ops import array_ops
from tensorflow.python.ops import control_flow_ops
from tensorflow.python.training import optimizer as tf_optimizer
from tensorflow.python.ops import variables as tf_variables

from tensorflow.python.ops import init_ops
from tensorflow.contrib import layers
from tensorflow.contrib.framework.python.ops import arg_scope
from tensorflow.contrib.layers.python.layers import layers as layers_lib
from tensorflow.python.ops import variable_scope

import os.path
import tensorflow.contrib.slim as slim
from tensorflow.contrib.slim.python.slim.nets import inception
import inception_preprocessing
import logging

# This is a bit of magic to make matplotlib figures appear inline in the notebook
# rather than in a new window.
%matplotlib inline
plt.rcParams['figure.figsize'] = (10.0, 8.0) # set default size of plots
plt.rcParams['image.interpolation'] = 'nearest'
plt.rcParams['image.cmap'] = 'gray'

# Some more magic so that the notebook will reload external python modules;
# see http://stackoverflow.com/questions/1907993/autoreload-of-modules-in-ipython
%load_ext autoreload
%autoreload 2

env: CUDA_VISIBLE_DEVICES=


In [2]:
DATASET_PATH = '/media/rs/0E06CD1706CD0127/Kapok/kaggle/'
PRETRAINED_MODEL_PATH = DATASET_PATH + 'Resnet/resnet2/resnet_v2_101.ckpt'
LOG_PATH = DATASET_PATH + 'Resnet/logs101/'
#TRAIN_PATH = DATASET_PATH + 'Split1/Train/'
TRAIN_PATH = '/media/rs/FC6CDC6F6CDC25E4/resample_dataset2/'
VAL_PATH = DATASET_PATH + 'Split1/Validation/'
TEST_PATH = DATASET_PATH + 'Test/'
CATEGORY_NAME_PATH = DATASET_PATH + 'category_names.csv'

In [3]:
def create_level_map():
    csv = pd.read_csv(CATEGORY_NAME_PATH).values
    level_list = [list(), list()]
    for row in csv: 
        for level in range(1,3):
            if row[level] not in level_list[level-1]:
                level_list[level-1].append(row[level])
    return dict(zip(level_list[0], range(len(level_list[0])))), dict(zip(level_list[1], range(len(level_list[1]))))

In [4]:
def cvt_csv2tfrecord():
    level0_map, level1_map = create_level_map()
    count = 0
    category_map = dict()
    category_level0_map = dict()
    category_level1_map = dict()
    csv = pd.read_csv(CATEGORY_NAME_PATH).values
    for row in csv:  
        category_id, level0, level1 = row[0], row[1], row[2]
        category_map[category_id] = count
        category_level0_map[int(category_id)] = level0_map[level0]
        category_level1_map[int(category_id)] = level1_map[level1]
        count += 1

    return category_map, category_level0_map, category_level1_map, len(level0_map), len(level1_map)

In [5]:
_category_map, _category_level0_map, _category_level1_map, _len_level0, _len_level1 = cvt_csv2tfrecord()
# print([ str(key) for key in _category_map.keys() ])
print([ str(key) for key in _category_level1_map.keys() ])

['1000018736', '1000005633', '1000013826', '1000005635', '1000005637', '1000013830', '1000005639', '1000013832', '1000020652', '1000006800', '1000005643', '1000005645', '1000013838', '1000005647', '1000005648', '1000005650', '1000005652', '1000005654', '1000019289', '1000005656', '1000005658', '1000005660', '1000005662', '1000005664', '1000005666', '1000005668', '1000005670', '1000005672', '1000013866', '1000005676', '1000005678', '1000005680', '1000005682', '1000005688', '1000005690', '1000005558', '1000005692', '1000005694', '1000020661', '1000019296', '1000005698', '1000005700', '1000000528', '1000005704', '1000005708', '1000005710', '1000005712', '1000005714', '1000005716', '1000005718', '1000000561', '1000013912', '1000005722', '1000005724', '1000005726', '1000005728', '1000005730', '1000014331', '1000005732', '1000005734', '1000005736', '1000005738', '1000005740', '1000013842', '1000013934', '1000005744', '1000000349', '1000005746', '1000005750', '1000005752', '1000005754', '1000

In [ ]:
def train_step():
    with tf.variable_scope("test", reuse=False):
        v = tf.get_variable('v', shape=[1], dtype=tf.int32, initializer = tf.constant_initializer(0))
        with ops.control_dependencies([tf.assign_add(v, tf.ones([1], tf.int32))]):
            barrier = control_flow_ops.no_op(name='update_barrier')

        b = tf.constant([2, 3], name="b")
        c = v + b
        return control_flow_ops.with_dependencies([barrier], c), v
def test_step():
    with tf.variable_scope("test", reuse=True):
        v = tf.get_variable('v', shape=[1], dtype=tf.int32, initializer = tf.constant_initializer(0))
        b = tf.constant([2, 3], name="b")
        c = v + b
        return c, v

train_c, train_v = train_step()
test_c, test_v = test_step()
init_op = tf.group(tf.global_variables_initializer())
with tf.Session() as sess:
    sess.run(init_op)
    for step in range(100):
        if step % 5 == 0:
            result_test_c, result_test_v = sess.run([test_c, test_v])
            print('test', result_test_c, result_test_v)
        else:
            result_train_c, result_train_v = sess.run([train_c, train_v])
            print('train', result_train_c, result_train_v)